In [ ]:
import tensorflow as tf 

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Flatten,Dense,Dropout

In [ ]:
# path to your dataset
path = '/content/drive/MyDrive/Face_mask_train/train'

In [ ]:
folders = os.listdir(path)
folders

['without_mask', 'with_mask']

In [ ]:
#initializations
data = []
#looping through all the folders
for folder in folders: 
    new_path = os.path.join(path,folder)
    total = len(os.listdir(new_path))
    label = folders.index(folder)
    print('folder_name = {} and no of images = {}'.format(folder,total))
    for img in os.listdir(new_path):#loop through all the images in that folder
    #read the images one by one
      try:
          img = cv2.imread( os.path.join(new_path,img),0) #grayscale
          img = cv2.resize(img,(100,100)) #check here once
          data.append([img,label])
      except Exception as e:
          print('Exception ',e)

folder_name = without_mask and no of images = 657
folder_name = with_mask and no of images = 658


In [ ]:
#shuffle the data in place
import random
random.shuffle(data)

In [ ]:
#seperating images are labels after shuffling
images = []
labels = []
for image,label in data:
    images.append(image)
    labels.append(label)

In [ ]:
#normalizing the images 
images = np.array(images)/255
labels = np.array(labels)

In [ ]:
images.shape

(1315, 100, 100)

In [ ]:
#converting to 4D 
images = np.expand_dims(images,axis = 3)


In [ ]:
images.shape

(1315, 100, 100, 1)

In [ ]:
# seperating train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(images,labels,test_size = 0.1,random_state = 0)

In [ ]:
#initializing the model
model=Sequential()

#The first CNN layer followed by Relu and MaxPooling layers
model.add(Conv2D(200,(3,3),activation = 'relu',input_shape=images.shape[1:]))
model.add(MaxPooling2D())

#The second convolution layer followed by Relu and MaxPooling layers
model.add(Conv2D(100,(3,3),activation = 'relu'))
model.add(MaxPooling2D())

#Flattening 
model.add(Flatten())

#Adding dropout to avoid overfitting
model.add(Dropout(0.5))

#Dense Layer of 50 neurons
model.add(Dense(50,activation='relu'))

#Output layer
model.add(Dense(1,activation='sigmoid'))

In [ ]:
#summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0         
_________________________________________________________________
dropout (Dropout)            (None, 52900)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [ ]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [ ]:
# BATCH_SIZE = 32
EPOCHS = 10
history=model.fit(X_train,y_train,epochs=EPOCHS, validation_data = (X_test,y_test)) #,callbacks=[checkpoint]

Epoch 1/10
37/37 [==============================] - 36s 74ms/step - loss: 0.7404 - accuracy: 0.5137 - val_loss: 0.6899 - val_accuracy: 0.6742
Epoch 2/10
37/37 [==============================] - 2s 44ms/step - loss: 0.6714 - accuracy: 0.6079 - val_loss: 0.5505 - val_accuracy: 0.6742
Epoch 3/10
37/37 [==============================] - 2s 44ms/step - loss: 0.5184 - accuracy: 0.7759 - val_loss: 0.3588 - val_accuracy: 0.8712
Epoch 4/10
37/37 [==============================] - 2s 44ms/step - loss: 0.3832 - accuracy: 0.8269 - val_loss: 0.3227 - val_accuracy: 0.8485
Epoch 5/10
37/37 [==============================] - 2s 44ms/step - loss: 0.2972 - accuracy: 0.8850 - val_loss: 0.2195 - val_accuracy: 0.9167
Epoch 6/10
37/37 [==============================] - 2s 43ms/step - loss: 0.2451 - accuracy: 0.9095 - val_loss: 0.1875 - val_accuracy: 0.9318
Epoch 7/10
37/37 [==============================] - 2s 44ms/step - loss: 0.1815 - accuracy: 0.9381 - val_loss: 0.1307 - val_accuracy: 0.9394
Epoch 8/10
3

In [ ]:
model.save('/content/drive/My Drive/FACE MASK/mask.h5')
